In [4]:
import numpy as np
import BiLinearIso4Nos as BL4
import sympy as sp
import meshio
import GeraBarraGmsh

In [5]:
#Entrada de Dados 
#Nesses Dics devemos ter os nomes das propriedades iguais dos nossos Physical Groups
Materiais = {"MAT1":[1e9,0.3]}
#Nome do Apoio(igual do Physical Group:"direção a restringir o movimento(x,y ou engaste(ambos))")
apoios = {"Sx":"x","Sy":"y"}
#Carregamento em Face: "Nome da Carga(igual do Physical Group", direção:1= normal, 2 = tangencial,valor]
cargas = [("LOAD",1,1000)]

#Gera Malha
GeraBarraGmsh.gerar_barra_tracao("barraTracao3.msh",L=1,h=0.1,nx=100,ny=10)

#Retorna as propriedades
nn, XY, ne, IJ, MAT, ESP, na, AP, nc, P = GeraBarraGmsh.processar_malhappp("barraTracao3.msh",Materiais,apoios,cargas)


In [6]:

#Monta Rigidez Global
K_b = BL4.RigidezGlobal(nn,ne,MAT,ESP,XY,IJ)
#Monta Força Global
F_b = BL4.ForcapGlobal(nn,ESP,XY,IJ,nc,P)
#Aplica as CChs homogêneas
K_mod,F_mod = BL4.AplicaCCH(nn,na,AP,K_b,F_b)

#Calcula o Deslocamento
U = np.linalg.solve(K_mod,F_mod)

#Calcula a Tensão ao longo dos elementos
Sigma = BL4.CalculaTensaoMalha(nn,ne,MAT,ESP,XY,IJ,U)

#Calcula Tensão de Von Mises
TVM = BL4.TensaoVonMises2D(ne,Sigma)

In [7]:
#Gera Malha com resultados
GeraBarraGmsh.export_to_gmsh_post("BarraResultado3.msh", XY, U, TVM,IJ )